In [1]:
import requests
import json
import math
from nba_api.stats.endpoints import *
import pandas
from scipy import stats

betano_url_games = []
team_stats_dict = {}


def get_content_betano(game_url):
    r = requests.get(game_url)
    content = r.text
    return content


def get_bet_event(game_url):
    content = get_content_betano(game_url)
    token_mark = '<script>window["initial_state"]='
    inx_start = content.find(token_mark) + len(token_mark)
    inx_end = content.find("</script>", inx_start)
    json_content = content[inx_start:inx_end]
    dic_data = json.loads(json_content)
    bet_event = dic_data["data"]["event"]
    return bet_event


def get_players(event):
    match = event["name"]
    separator = " - "
    player1, player2 = str.split(match, separator)
    return player1, player2


def find_by_name(value, lst):
    return list(filter(lambda x: x["name"] == value, lst))


def get_bet_target(token_class, bet_player, markets):
    winner_markets = find_by_name(token_class, markets)
    if len(winner_markets) > 0:
        winner_market = winner_markets[0]
    else:
        raise Exception(f"ERROR: {token_class} not found")

    tracked_player_list = find_by_name(bet_player, winner_market["selections"])
    if len(tracked_player_list) > 0:
        tracked_player_obj = tracked_player_list[0]
    else:
        raise Exception(f"ERROR: {bet_player} not found")
    return tracked_player_obj


def find_by_name(value, lst):
    return list(filter(lambda x: x["name"] == value, lst))


def get_bet_target(token_class, bet_player, markets):
    winner_markets = find_by_name(token_class, markets)

    if len(winner_markets) > 0:
        winner_market = winner_markets[0]
    else:
        print(f"ERROR: {token_class} not found")
        return

    tracked_player_list = find_by_name(bet_player, winner_market["selections"])
    if len(tracked_player_list) > 0:
        tracked_player_obj = tracked_player_list[0]
    else:
        raise Exception(f"ERROR: {bet_player} not found")
    return tracked_player_obj


def has_triggered_bet(tracked_player_obj, bet_value_expected):
    if tracked_player_obj["price"] >= bet_value_expected:
        msg = f'$ {tracked_player_obj["name"]} alcancou {bet_value_expected}: Atual {tracked_player_obj["price"]}'
        print(msg)
        return msg


def is_second_half(event):
    try:
        period = event["liveData"]["periodDescription"]
        if period != "P1" and period != "P2":
            return True
    except:
        return


def can_bet_for_total_points_on_2nd_half(event):
    if is_second_half(event):
        first = event["liveData"]["results"]["firstPeriodScore"]
        second = event["liveData"]["results"]["secondPeriodScore"]
        firstHome, firstAway = first["home"], first["away"]
        secondHome, secondAway = second["home"], second["away"]
        totalFirstHalf = (
            int(firstHome) + int(firstAway) + int(secondHome) + int(secondAway)
        )
        markets = event["markets"]
        total_points = find_by_name("Total de Pontos", markets)
        if len(total_points) == 0:
            return
        less_than_points_obj = total_points[0]["selections"][1]
        handicap = less_than_points_obj["handicap"]
        price = less_than_points_obj["price"]
        projectedPoints = totalFirstHalf * 2
        can_bet = projectedPoints + 6 < handicap
        if can_bet == "YES":
            msg = f"$ JOGAR TOTAL DE PONTOS 2o Tempo:\n{handicap}->{price}/esperado {projectedPoints}"
            print(msg)
            return msg


class TeamStats:
    def __init__(self, team_name, pontos):
        cut_section = 4
        cut_pnts = len(pontos) // cut_section
        self.name = team_name
        self.media = round(pontos.mean(),1)
        self.std = round(pontos.std(),1)
        self.media_ultima = round(pontos[:cut_pnts].mean(),1)
        self.hot_mean = round(pontos.sort_values(ascending=False)[:cut_pnts].mean(),1)
        self.hot_std = round(pontos.sort_values(ascending=False)[:cut_pnts].std(),1)
        self.cold_mean = round(pontos.sort_values()[:cut_pnts].mean(),1)
        self.cold_std = round(pontos.sort_values()[:cut_pnts].std(),1)
        self.pontos = pontos

    
    # def pnts_esperados(self):
    #     res = (self.hot + self.media_ultima) / 2
    #     return math.floor(res)

    # def pnts_esperados_baixo(self):
    #     res = (self.cold + self.media_ultima) / 2
    #     return math.floor(res)

    def __str__(self):
        return f"{self.name}:{self.pnts_esperados_baixo()}<-->{self.pnts_esperados()}"


def get_team_stats(team):
    teams_data = leaguedashteamstats.LeagueDashTeamStats().get_data_frames()[0]
    team_id_res = teams_data[teams_data["TEAM_NAME"] == team]["TEAM_ID"]
    team_id = team_id_res.values[0]
    pontos = teamgamelogs.TeamGameLogs(
        season_nullable="2023-24", team_id_nullable=team_id
    ).get_data_frames()[0]["PTS"]
    return TeamStats(team, pontos)


def print_json(dic):
    print(json.dumps(dic))


import pandas as pd
from datetime import datetime as dt

ZEBRA_GAME_FILENAME = "zebras_game.csv"


def to_date_str(x):
    return str(x.date()).replace("-", "")


def get_betano_games():
    betano_games_url = "https://br.betano.com/sport/basquete/eua/nba/441g/"
    content = get_content_betano(betano_games_url)
    marca_inicio = '<script type="application/ld+json">'
    indice_inicio = content.find(marca_inicio) + len(marca_inicio)
    indice_fim = content.find("</script>", indice_inicio)
    json_content = content[indice_inicio:indice_fim]
    dic_data = json.loads(json_content)

    return dic_data


def find_zebras_game():
    betano_games = get_betano_games()
    zebra_games = []
    for game in betano_games:
        url = game["url"]
        evento = get_bet_event(url)
        markets = evento["markets"]

        aposta_vencedor = find_by_name("Vencedor", markets)
        if len(aposta_vencedor) == 0:
            continue

        apostas = aposta_vencedor[0]["selections"]
        player1 = apostas[0]
        player2 = apostas[1]
        zebra_threshold = 4.5
        if player1["price"] >= zebra_threshold or player2["price"] >= zebra_threshold:
            # print(f"! ZEBRA: {player1['name']} X {player2['name']} {url}")
            o = {}
            o["startTime"] = evento["startTime"]
            o["name"] = evento["name"]
            o["id"] = int(evento["id"])
            zebra = player1 if player1["price"] >= zebra_threshold else player2
            o["zebra_name"] = zebra["name"]
            o["zebra_price"] = zebra["price"]
            zebra_games.append(o)
    return zebra_games


def save_betano_zebras():
    zebra_games = find_zebras_game()

    startDateCol = "startDate"
    try:
        df_csv = pd.read_csv(ZEBRA_GAME_FILENAME, index_col="startDate")
    except:
        df_csv = pd.DataFrame(
            columns=[
                startDateCol,
                "startTime",
                "name",
                "id",
                "zebra_name",
                "zebra_price",
            ]
        )
        df_csv = df_csv.set_index("startDate")
    df = pandas.DataFrame()
    df2 = df.from_records(zebra_games)
    df_to_date = lambda x: dt.fromtimestamp(x / 1000)
    df2["startDate"] = [to_date_str(df_to_date(i)) for i in df2["startTime"].values]
    df2 = df2.set_index("startDate")

    ids_existent = df_csv.id.values
    df_csv_news = df2[~df2["id"].isin(ids_existent)]

    df_csv_save = pd.concat([df_csv, df_csv_news])
    df_csv_save.to_csv(ZEBRA_GAME_FILENAME)


def get_zebra_from_today():
    df_csv = pd.read_csv(ZEBRA_GAME_FILENAME)
    today = dt.today()
    today_inx = int(to_date_str(today))
    # df_zebra_today =  df_csv[df_csv['startDate']==today_inx]
    df_zebra_today = df_csv
    return df_zebra_today


save_betano_zebras()


def bet_pnts_by_game(p1_stats: TeamStats, p2_stats: TeamStats, markets):
    total_de_pontos = find_by_name("Total de Pontos", markets)
    if len(total_de_pontos) == 0:
        return
    menos_de_total_de_pontos = total_de_pontos[0]["selections"][1]
    aposta_tot_pnts = menos_de_total_de_pontos["handicap"]
    maior_pnts_do_jogo_provavel = p1_stats.hot_mean + p2_stats.hot_mean
    menor_pnts_do_jogo_provavel = p1_stats.cold_mean + p2_stats.cold_mean
    # tot_pnts_str = f"(!) TOT_PNTS: {math.floor(menor_pnts_do_jogo_provavel)}<-->{math.floor(maior_pnts_do_jogo_provavel)}"
    # print(tot_pnts_str)

    if aposta_tot_pnts > maior_pnts_do_jogo_provavel:
        preco = menos_de_total_de_pontos["price"]
        std_teams = math.sqrt(p1_stats.hot_std**2 + p2_stats.hot_std**2)
        probabilidade_hot = calcula_probabilidade(
            aposta_menor_que_curr, maior_pnts_do_jogo_provavel, std_teams
        )
        expectativa_hot = calcula_expectativa(probabilidade_hot, preco - 1)
        print(
            f"$ JOGAR TOT_PNTS APOSTA: {aposta_tot_pnts}->{preco}, Expec:R${expectativa_hot}, Prob:{probabilidade_hot}, ESPERADO {maior_pnts_do_jogo_provavel}"
        )
    total_de_pontos_alternativas = get_total_de_pontos_alternativas(
        markets, total_de_pontos
    )

    if not total_de_pontos_alternativas:
        return
    if type(total_de_pontos_alternativas) == list:
        rows = total_de_pontos_alternativas[0]["tableLayout"]["rows"]
    else:
        rows = total_de_pontos_alternativas["tableLayout"]["rows"]

    preco_maior_que = lambda x: x["groupSelections"][0]["selections"][0]
    for curr in rows[::-1]:
        maior_que_curr = preco_maior_que(curr)
        aposta_maior_que_curr = maior_que_curr["handicap"]
        if aposta_maior_que_curr < menor_pnts_do_jogo_provavel:
            preco = maior_que_curr["price"]
            std_teams = math.sqrt(p1_stats.cold_std**2 + p2_stats.cold_std**2)
            probabilidade_cold = calcula_probabilidade(
                aposta_maior_que_curr, menor_pnts_do_jogo_provavel, std_teams
            )
            expectativa_cold = calcula_expectativa(probabilidade_cold, preco - 1)
            print(
                f"$ JOGAR TOT_PNTS(ALT) MAIOR QUE: {aposta_maior_que_curr}->{preco}, Expec:R${expectativa_cold}, Prob:{probabilidade_cold}, ESPERADO MAIOR: {menor_pnts_do_jogo_provavel}"
            )
            break

    preco_menor_que = lambda x: x["groupSelections"][0]["selections"][1]
    for i in range(len(rows)):
        menor_que_curr = preco_menor_que(rows[i])
        aposta_menor_que_curr = menor_que_curr["handicap"]
        if aposta_menor_que_curr > maior_pnts_do_jogo_provavel:
            preco = menor_que_curr["price"]
            std_teams = math.sqrt(p1_stats.hot_std**2 + p2_stats.hot_std**2)
            probabilidade_hot = calcula_probabilidade(
                aposta_menor_que_curr, maior_pnts_do_jogo_provavel, std_teams
            )
            expectativa_hot = calcula_expectativa(probabilidade_hot, preco - 1)
            print(
                f"$ JOGAR TOT_PNTS(ALT) MENOR QUE: {aposta_menor_que_curr }->{preco}, Expec:R${expectativa_hot}, Prob:{probabilidade_hot}, ESPERADO MENOR: {maior_pnts_do_jogo_provavel}"
            )
            break


def bet_pnts_by_team(player_stats, markets):
    try:
        player_tot_pnts_markets = find_by_name(
            f"Total de Pontos {player_stats.name}", markets
        )
        player_tot_pnts = player_tot_pnts_markets[0]["selections"]
        if len(player_tot_pnts_markets) > 1:
            player_tot_pnts = player_tot_pnts + player_tot_pnts_markets[1]["selections"]
        else:
            player_tot_pnts_markets = find_by_name(
                f"Total de Pontos {player_stats.name} (alternativas)", markets
            )
            if len(player_tot_pnts_markets) > 0:
                player_tot_pnts = (
                    player_tot_pnts + player_tot_pnts_markets[0]["selections"]
                )

        player_tot_pnts = list(sorted(player_tot_pnts, key=lambda x: x["handicap"]))
    except:
        return

    menor_pnts_do_jogo_provavel_player = player_stats.cold_mean
    maior_pnts_do_jogo_provavel_player = player_stats.hot_mean
    player_tot_pnts_mais_de = [o for o in player_tot_pnts if "Mais de" in o["name"]][
        ::-1
    ]
    player_tot_pnts_menos_de = [o for o in player_tot_pnts if "Menos de" in o["name"]]

    for i in range(len(player_tot_pnts_mais_de)):
        maior_que_curr = player_tot_pnts_mais_de[i]
        aposta_maior_que_curr = maior_que_curr["handicap"]
        if aposta_maior_que_curr < menor_pnts_do_jogo_provavel_player:
            preco = maior_que_curr["price"]
            probabilidade_cold = calcula_probabilidade(
                aposta_maior_que_curr, player_stats.cold_mean, player_stats.cold_std
            )
            expectativa_cold = calcula_expectativa(probabilidade_cold, preco - 1)
            print(
                f"$ JOGAR PNTS(ALT){player_stats.name} MAIOR QUE: {aposta_maior_que_curr}->{preco}, expec:R${expectativa_cold},Prob:{probabilidade_cold}, ESPERADO MAIOR: {menor_pnts_do_jogo_provavel_player}"
            )
            break
    for i in range(len(player_tot_pnts_menos_de)):
        menor_que_curr = player_tot_pnts_menos_de[i]
        aposta_menor_que_curr = menor_que_curr["handicap"]
        if aposta_menor_que_curr > maior_pnts_do_jogo_provavel_player:
            preco = menor_que_curr["price"]
            probabilidade_hot = calcula_probabilidade(
                aposta_menor_que_curr, player_stats.hot_mean, player_stats.hot_std
            )
            expectativa_hot = calcula_expectativa(probabilidade_hot, preco - 1)
            print(
                f"$ JOGAR PNTS(ALT){player_stats.name} MENOR QUE: {aposta_menor_que_curr }->{preco}, expec:R${expectativa_hot},Prob:{probabilidade_hot}, ESPERADO MENOR: {maior_pnts_do_jogo_provavel_player}"
            )
            break
    return player_tot_pnts


def get_total_de_pontos_alternativas(markets, total_de_pontos):
    total_de_pontos_alternativas = total_de_pontos
    if len(total_de_pontos) > 1:
        total_de_pontos_alternativas = total_de_pontos[1]
    else:
        total_de_pontos_alternativas = find_by_name(
            "Total de Pontos (alternativas)", markets
        )
        if len(total_de_pontos_alternativas) > 0:
            total_de_pontos_alternativas += total_de_pontos_alternativas[0]
    return total_de_pontos_alternativas


def calcula_expectativa_time(time_stats, ponto_aposta, ganho_no_jogo):
    probabilidade = calcula_probabilidade_de_pontos_do_time(ponto_aposta, time_stats)
    return calcula_expectativa(probabilidade, ganho_no_jogo)


def calcula_probabilidade_de_pontos_do_time(alvo, time_stats):
    return calcula_probabilidade(alvo, time_stats.media, time_stats.pontos.std())


def calcula_probabilidade(alvo, media, desvio_padrao):
    return round(stats.norm.cdf((alvo - media) / desvio_padrao), 2)


def calcula_expectativa(probabilidade, ganho_no_jogo, perda_no_jogo=1):
    return round(ganho_no_jogo * probabilidade - perda_no_jogo * (1 - probabilidade), 2)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9960\2354785310.py:235: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df_csv_save = pd.concat([df_csv, df_csv_news])


In [2]:
def get_today_betano_games_urls():
    betano_games = get_betano_games()
    todays_game_url = []
    today = dt.today().date()
    for game in betano_games:
        time_obj_game = dt.fromisoformat(game['startDate'])
        date_game  = time_obj_game.astimezone().date()
        if date_game == today:
            todays_game_url.append(game['url']+'?bt=8')
        else:
            game_event = get_bet_event(game["url"])
            try:
                if game_event["liveNow"]:
                    todays_game_url.append(game['url']+'?bt=8')
            except:
                pass

        # break
    return todays_game_url

In [3]:
import time
from plyer import notification
import pygame
import ctypes  # An included library with Python install.
import winsound

# Inicializa o mixer do pygame para sons
pygame.mixer.init()

def enviar_alerta(msg, game ):
    # Exibe um popup
    # notification.notify(
    #     title='Alerta',
    #     message=msg,
    #     app_name='Temporizador'
    # )
    
    Mbox.BoxOkCancelAsync(f"ALERTA BET {game}", msg)

    # Toca um som de alerta
    pygame.mixer.music.load('alarm.mp3')
    pygame.mixer.music.play()


class Mbox:
    OK = 1
    CANCELADO = 2

    def BoxOkCancel(title, text):
        winsound.MessageBeep(winsound.MB_OK)
        ctypes.windll.user32.MessageBoxW(0, text, title, 1)

    def Alerta(title, text=""):
        winsound.MessageBeep(winsound.MB_OK)
        funcao = lambda: ctypes.windll.user32.MessageBoxW(0, text, title, 0x1000)
        Mbox.executar_alerta_paralelo(funcao)

    def executar_alerta_paralelo(funcao_alerta, *args):
        thread_exec = ThreadSimples(funcao_alerta, *args)
        thread_exec.start()

    def BoxOkCancelAsync(title, text):
        Mbox.executar_alerta_paralelo(Mbox.BoxOkCancel, title, text)


pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
import threading
import time
TEMPO_ALARME   =  30 # SEGUNDOS
class ThreadCustom(threading.Thread):
    def __init__(self, tempo, executar_funcao):
        super(ThreadCustom, self).__init__()
        self.parar = threading.Event()
        self.tempo = tempo
        self.executar = executar_funcao

    def run(self):
        while not self.parar.is_set():
            self.executar()
            time.sleep(self.tempo)

    def parar_execucao(self):
        print("Parando a execução...")
        self.parar.set()

def parar_thread(thread):
    thread.parar_execucao()
    thread.join()  # Aguarde até que a thread termine completamente

class ThreadSimples(threading.Thread):
    def __init__(self, funcao, *args, **kwargs):
        # If you are going to have your own constructor, make sure you call the parent constructor too!
        #        Thread.__init__(self)
        super().__init__()
        self.funcao = funcao
        self.kwargs = kwargs
        self.args = args

    def run(self):
        self.funcao(*self.args, **self.kwargs)

In [5]:


betano_url_games = get_today_betano_games_urls()
zebras = get_zebra_from_today()

def pre_game_routine():
    for url in betano_url_games:
        print(">>>>")
        process_game(url)

def process_game(game_url):
    try:
        event = get_bet_event(game_url)
    except:
        print(f"ERROR ON: {game_url}")
        return
    player1_name, player2_name = get_players(event)

    if player1_name in team_stats_dict:
        p1_stats = team_stats_dict[player1_name]
    else:
        p1_stats = get_team_stats(player1_name)
        team_stats_dict[player1_name] = p1_stats

    if player2_name in team_stats_dict:
        p2_stats = team_stats_dict[player2_name]
    else:
        p2_stats = get_team_stats(player2_name)
        team_stats_dict[player2_name] = p2_stats
    
    if "liveNow" in event:
        datetime_info = "LIVE" 
    else:
        datetime_info = dt.fromtimestamp(event["startTime"] / 1000).astimezone()


    print(f">{datetime_info} {player1_name} X {player2_name}: {game_url}")
    bet_value_expected = 3
    game_name = event["shortName"]
    markets = event["markets"]
    token_winner = "Vencedor"
    zebras_curr = zebras[zebras["name"]==game_name]
    zebras_name = zebras_curr.zebra_name.values
    p2_is_zebra  = player2_name in zebras_name
    p1_is_zebra  = player1_name in zebras_name
    if p1_is_zebra or p2_is_zebra:
        track_favorite = player2_name if p1_is_zebra else player1_name
        tracked_player_favorite_obj = get_bet_target(token_winner, track_favorite, markets)
        if tracked_player_favorite_obj is None:
            return
    # print(f'{token_winner}:{tracked_player_obj["name"]}:{tracked_player_obj["price"]}')
        result = has_triggered_bet(tracked_player_favorite_obj, bet_value_expected)
        if result:
            enviar_alerta(result, game_name)

    bet_on_2nd_half = can_bet_for_total_points_on_2nd_half(event)
    if bet_on_2nd_half:
        enviar_alerta(bet_on_2nd_half, game_name)

    bet_pnts_by_game(p1_stats, p2_stats, markets)
    bet_pnts_by_team( p1_stats, markets)
    bet_pnts_by_team( p2_stats, markets)


pre_game_routine()
# process_game("https://br.betano.com/odds/boston-celtics-denver-nuggets/44146498/?bt=8")

>>>>
>2024-01-20 17:10:00-03:00 Detroit Pistons X Milwaukee Bucks: https://br.betano.com/odds/detroit-pistons-milwaukee-bucks/44189304/?bt=8
$ JOGAR TOT_PNTS(ALT) MENOR QUE: 264.5->1.17, Expec:R$-0.4, Prob:0.51, ESPERADO MENOR: 264.1
$ JOGAR PNTS(ALT)Detroit Pistons MENOR QUE: 125.5->1.18, expec:R$-0.37,Prob:0.53, ESPERADO MENOR: 124.8
$ JOGAR PNTS(ALT)Milwaukee Bucks MENOR QUE: 139.5->1.17, expec:R$-0.4,Prob:0.51, ESPERADO MENOR: 139.3
>>>>
>2024-01-20 21:10:00-03:00 Charlotte Hornets X Philadelphia 76ers: https://br.betano.com/odds/charlotte-hornets-philadelphia-76ers/44189322/?bt=8
>>>>
>2024-01-20 21:10:00-03:00 Washington Wizards X San Antonio Spurs: https://br.betano.com/odds/washington-wizards-san-antonio-spurs/44189336/?bt=8
$ JOGAR TOT_PNTS(ALT) MENOR QUE: 257.5->1.23, Expec:R$-0.34, Prob:0.54, ESPERADO MENOR: 256.79999999999995
$ JOGAR PNTS(ALT)Washington Wizards MENOR QUE: 129.5->1.25, expec:R$-0.32,Prob:0.54, ESPERADO MENOR: 129.2
$ JOGAR PNTS(ALT)San Antonio Spurs MENOR QU

In [16]:
team_stats_dict['Detroit Pistons'].hot_mean + team_stats_dict['Milwaukee Bucks'].hot_mean

260.0

In [ ]:

if len(betano_url_games) ==0:
    betano_url_games = get_today_betano_games_urls()
    
def live_routine():
    for url in betano_url_games[:5]:
        print(">>>>")
        check_signals_live(url)

def check_signals_live(game_url):
    try:
        event = get_bet_event(game_url)
    except:
        print(f"ERROR ON: {game_url}")
        return
    player1_name, player2_name = get_players(event)
    p1_stats = get_team_stats(player1_name)
    p2_stats = get_team_stats(player2_name)
    pnts_do_jogo_provavel = p1_stats.pnts_esperados() + p2_stats.pnts_esperados()
    menor_pnts_do_jogo_provavel = (
        p1_stats.pnts_esperados_baixo() + p2_stats.pnts_esperados_baixo()
    )
    tot_pnts_str = f"TOT_PNTS: {math.floor(menor_pnts_do_jogo_provavel)}<-->{math.floor(pnts_do_jogo_provavel)}"
    print(f"> {player1_name} X {player2_name}: {tot_pnts_str}: {game_url}")
    bet_value_expected = 3
    markets = event["markets"]
    token_winner = "Vencedor"
    zebras = get_zebra_from_today()
    zebras_name = zebras.zebra_name.values
    p2_is_zebra  = player2_name in zebras_name
    p1_is_zebra  = player1_name in zebras_name
    if p1_is_zebra or p2_is_zebra:
        track_favorite = player2_name if p1_is_zebra else player1_name
        tracked_player_favorite_obj = get_bet_target(token_winner, track_favorite, markets)
        
    # print(f'{token_win ner}:{tracked_player_obj["name"]}:{tracked_player_obj["price"]}')
        result = has_triggered_bet(tracked_player_favorite_obj, bet_value_expected)
        if result:
            enviar_alerta(result,event["shortName"])

    bet_on_2nd_half = can_bet_for_total_points_on_2nd_half(event)
    if bet_on_2nd_half:
        enviar_alerta(bet_on_2nd_half, event["shortName"])

    # total_de_pontos = find_by_name("Total de Pontos", markets)
    # menos_de_total_de_pontos = total_de_pontos[0]["selections"][1]
    # aposta_tot_pnts = menos_de_total_de_pontos["handicap"]

    # if aposta_tot_pnts > pnts_do_jogo_provavel:
    #     preco = menos_de_total_de_pontos["price"]
    #     print(f"$ JOGAR TOT_PNTS APOSTA: {aposta_tot_pnts}->{preco}")
    # try:
    #     total_de_pontos_alternativas = total_de_pontos[1]
    # except:
    #     total_de_pontos_alternativas = find_by_name(
    #         "Total de Pontos (alternativas)", markets
    #     )[0]
    # rows = total_de_pontos_alternativas["tableLayout"]["rows"]

    # preco_maior_que = lambda x: x["groupSelections"][0]["selections"][0]
    # for i in range(len(rows)):
    #     maior_que_curr = preco_maior_que(rows[i])
    #     aposta_maior_que_curr = maior_que_curr["handicap"]
    #     if aposta_maior_que_curr + 1 > menor_pnts_do_jogo_provavel:
    #         preco = maior_que_curr["price"]
    #         print(f"$ JOGAR TOT_PNTS(ALT) MAIOR QUE: {aposta_maior_que_curr}->{preco}/ESPERADO MAIOR: {menor_pnts_do_jogo_provavel}")
    #         break

    # preco_menor_que = lambda x: x["groupSelections"][0]["selections"][1]
    # for i in range(len(rows)):
    #     menor_que_curr = preco_menor_que(rows[i])
    #     aposta_menor_que_curr = menor_que_curr["handicap"]
    #     if aposta_menor_que_curr > pnts_do_jogo_provavel:
    #         preco = menor_que_curr["price"]
    #         print(f"$ JOGAR TOT_PNTS(ALT) MENOR QUE: {aposta_menor_que_curr }->{preco}/ESPERADO MENOR: {pnts_do_jogo_provavel}")
    #         break

    # bet_pnts_by_team( p1_stats, markets)
    # bet_pnts_by_team( p2_stats, markets)
    

live_routine()

>>>>
> Boston Celtics X Denver Nuggets: TOT_PNTS: 224<-->254: https://br.betano.com/live/boston-celtics-denver-nuggets/44146498/?bt=8
>>>>
> Miami Heat X Atlanta Hawks: TOT_PNTS: 212<-->242: https://br.betano.com/live/miami-heat-atlanta-hawks/44146507/?bt=8
>>>>
> New Orleans Pelicans X Phoenix Suns: TOT_PNTS: 221<-->250: https://br.betano.com/live/new-orleans-pelicans-phoenix-suns/44146517/?bt=8


In [ ]:

# thread = ThreadCustom(TEMPO_ALARME,live_routine)
# thread.start()